<a href="https://www.kaggle.com/code/devilbornfire/chatbot-nlp?scriptVersionId=170776846" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> PROJECT--- BUILDING NLP BASED CHATBOT USING NLTK

In [ ]:
#installation 

!pip install pdfminer.six
!pip install fpdf

In [ ]:
#importing libraries 

import numpy as np
import pandas as pd
import string
import nltk
import random 

    LOADING DATASET

In [ ]:
df=pd.read_csv("/kaggle/input/text-document-classification-dataset/df_file.csv")
df

    DATA ANALYTICS:

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#value_counts 

df['Text'].value_counts()

    CONVERTING TEXT TO PDF:

In [ ]:
#using FPDF

from fpdf import FPDF
pdf=FPDF()
pdf.add_page()
pdf.set_font("Arial",size=12)


for index,row in df.iterrows():
    pdf.cell(200,10,txt=str(row),ln=True,align='L')
    

pdf.output("output.pdf")

EXTRACTING TEXT FROM PDF:

In [ ]:
#using pdfminer

from pdfminer.high_level import extract_text 
text=extract_text('output.pdf')
print(repr(text))
print(text)

In [ ]:
#downloading

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

    TOKENIZATION:

In [ ]:

sentence_tokens=nltk.sent_tokenize(text)
word_tokens=nltk.word_tokenize(text)

In [ ]:
print(sentence_tokens)

In [ ]:
print(word_tokens)

    LEMMATIZATION:

In [ ]:
nltk.download('wordnet')
lemm=nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemm.lemmatize(token) for token in tokens]

remove_punct_dict=dict((ord(punct),None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

    CHATBOT INTERFACE:

In [ ]:
greet_inputs=('hello','hi','whassup','namaste','kemchoo','how are you')
greet_responses=('hi','hey there',' i am fine', 'good','me too')
def greet(sentence):
    for word in sentence.split():
        if word.lower() in greet_inputs:
            return random.choice(greet_responses)

TfIDF VECTORIZER:

In [ ]:
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

    BUILDING CHATFLOW:

In [ ]:
def response(user_response):
    robot_response= ' '
    TfidVec=TfidfVectorizer(tokenizer=LemNormalize,stop_words='english')
    tfidf=TfidVec.fit_transform(sentence_tokens)
    vals=cosine_similarity(tfidf[-1],tfidf)
    idx=vals.argsort()[0][-2]
    flat=vals.flatten()
    flat.sort()
    req_tfidf=flat[-2]
    if(req_tfidf==0):
        robot_response=robot_response+ "I am sorry.Unable to understand!"
        return robot_response
    else:
        robot_response=robot_response+sentence_tokens[idx]
        return robot_response

    RESPONSE GENERATION:

In [ ]:
flag = True 
print("Hello I am retreivel learning chatbot please type your text after greeting to me for ending convo type bye!")
while(flag==True):
    user_response=input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thank you' or user_response=='thanks'):
            flag=True
            print('Bot: You are welcome!')
        else:
            if(greet(user_response) !=None):
               print('Bot:' + greet(user_response))
            else:
               sentence_tokens.append(user_response)
               word_tokens=nltk.word_tokenize(user_response)
               final_words=list(set(word_tokens))
               print('Bot:',end='')
               print(response(user_response))
               sentence_tokens.remove(user_response)
    else:
               flag = False
               print('Bot: Goodbye!')